In [20]:
# !pip install catboost==0.15.2
# !pip install tqdm
# !pip install xlrd
import numpy as np
import pandas as pd
import catboost as cbt
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss
import gc
import math
import time
from tqdm import tqdm
import datetime
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import os
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [21]:
train = pd.read_csv('first_round_training_data.csv')
test = pd.read_csv('first_round_testing_data.csv')
submit = pd.read_csv('submit_example.csv')
data = train.append(test).reset_index(drop=True)
dit = {'Excellent':0,'Good':1,'Pass':2,'Fail':3}
data['label'] = data['Quality_label'].map(dit)
train['label'] = train['Quality_label'].map(dit)

In [22]:
feature_name = ['Parameter{0}'.format(i) for i in range(5, 11)]
tr_index = ~data['label'].isnull()
X_train = data[tr_index][feature_name].reset_index(drop=True)
y = data[tr_index]['label'].reset_index(drop=True).astype(int)
X_test = data[~tr_index][feature_name].reset_index(drop=True)

print(X_train.shape,X_test.shape)
oof = np.zeros((X_train.shape[0],4))
# np.zeros的作用是返回一个给定形状和类型的用0填充的数组
# 这个shape[0]就是一共多少行，这里一共6000行
# 打印出来看看就知道了
# 他这一个应该是提前先画出来一个矩阵
prediction = np.zeros((X_test.shape[0],4))
seeds = [19970412, 2019 * 2 + 1024, 4096, 2048, 1024]
# random_state :默认None，当 shuffle为True的时候使用。
# 如果是int，表示随机数生成器使用的种子，为None时，是np.random随机使用的实例。
num_model_seed = 5
for model_seed in range(num_model_seed):
    print(model_seed + 1)
    oof_cat = np.zeros((X_train.shape[0],4))
    prediction_cat=np.zeros((X_test.shape[0],4))
    skf = StratifiedKFold(n_splits=5, random_state=seeds[model_seed], shuffle=True)
    # shuffle:是否在每次分割之前打乱顺序
    # StratifiedKFold用法类似Kfold，但是他是分层采样，确保训练集，测试集中各类别样本的比例与原始数据集中相同
    for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
        print(index)
        train_x, test_x, train_y, test_y = X_train.iloc[train_index], X_train.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        gc.collect()
        cbt_model = cbt.CatBoostClassifier(iterations=800,learning_rate=0.01,verbose=300,
                                        #    verbose是显示不显示详细信息的意思
                                        early_stopping_rounds=200,task_type='GPU',
                                        loss_function='MultiClass')
        cbt_model.fit(train_x, train_y ,eval_set=(train_x, train_y))
        oof_cat[test_index] += cbt_model.predict_proba(test_x)
        prediction_cat += cbt_model.predict_proba(X_test)/5
        gc.collect()
    oof += oof_cat / num_model_seed
    prediction += prediction_cat / num_model_seed
    print('logloss',log_loss(pd.get_dummies(y).values, oof_cat))
    print('ac',accuracy_score(y, np.argmax(oof_cat,axis=1)))
    print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[y] - oof_cat))/480))
print('logloss',log_loss(pd.get_dummies(y).values, oof))
print('ac',accuracy_score(y, np.argmax(oof,axis=1)))
print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[y] - oof))/480))

(6000, 6) (6000, 6)
1
0
0:	learn: -1.3808396	test: -1.3808394	best: -1.3808394 (0)	total: 18.9ms	remaining: 15.1s
300:	learn: -1.0728196	test: -1.0728196	best: -1.0728196 (300)	total: 3.42s	remaining: 5.67s
600:	learn: -1.0367846	test: -1.0367846	best: -1.0367846 (600)	total: 6.76s	remaining: 2.24s
799:	learn: -1.0199881	test: -1.0199881	best: -1.0199881 (799)	total: 9.02s	remaining: 0us
bestTest = -1.019988085
bestIteration = 799
Shrink model to first 800 iterations.
1
0:	learn: -1.3809463	test: -1.3809464	best: -1.3809464 (0)	total: 13.3ms	remaining: 10.6s
300:	learn: -1.0747083	test: -1.0747083	best: -1.0747083 (300)	total: 3.38s	remaining: 5.6s
600:	learn: -1.0399395	test: -1.0399396	best: -1.0399396 (600)	total: 6.56s	remaining: 2.17s
799:	learn: -1.0237216	test: -1.0237217	best: -1.0237217 (799)	total: 8.67s	remaining: 0us
bestTest = -1.023721737
bestIteration = 799
Shrink model to first 800 iterations.
2
0:	learn: -1.3808887	test: -1.3808885	best: -1.3808885 (0)	total: 12.2ms	re

bestTest = -1.023043498
bestIteration = 799
Shrink model to first 800 iterations.
3
0:	learn: -1.3809387	test: -1.3809387	best: -1.3809387 (0)	total: 12.4ms	remaining: 9.92s
300:	learn: -1.0753827	test: -1.0753828	best: -1.0753828 (300)	total: 3.18s	remaining: 5.28s
600:	learn: -1.0414800	test: -1.0414800	best: -1.0414800 (600)	total: 6.32s	remaining: 2.09s
799:	learn: -1.0251646	test: -1.0251647	best: -1.0251647 (799)	total: 8.41s	remaining: 0us
bestTest = -1.02516474
bestIteration = 799
Shrink model to first 800 iterations.
4
0:	learn: -1.3808619	test: -1.3808620	best: -1.3808620 (0)	total: 11.7ms	remaining: 9.34s
300:	learn: -1.0732786	test: -1.0732784	best: -1.0732784 (300)	total: 3.4s	remaining: 5.63s
600:	learn: -1.0380015	test: -1.0380015	best: -1.0380015 (600)	total: 6.63s	remaining: 2.19s
799:	learn: -1.0209641	test: -1.0209641	best: -1.0209641 (799)	total: 8.83s	remaining: 0us
bestTest = -1.020964064
bestIteration = 799
Shrink model to first 800 iterations.
logloss 1.08353121

In [23]:
sub = test[['Group']]
prob_cols = [i for i in submit.columns if i not in ['Group']]
for i, f in enumerate(prob_cols):
    sub[f] = prediction[:, i]
for i in prob_cols:
    sub[i] = sub.groupby('Group')[i].transform('mean')
sub = sub.drop_duplicates()
sub.to_csv("submission.csv",index=False)